Define these constants:

In [34]:
LOAD_FILES_CAP = None  # for testing load only some files (loading slow)
# LOAD_FILES_CAP = None
output_root = '/Users/adam/pschool/bakalarka/cryptic_output/output'  # unzipped tree with json outputs

In [35]:
import json
from pathlib import Path
from typing import Iterable

import pandas as pd


def load_df(json_files: Iterable[Path]):
    super_df = pd.DataFrame()

    json_files = list(json_files)
    if LOAD_FILES_CAP:
        json_files = list(json_files)[:LOAD_FILES_CAP]  # testing hack

    total_files = len(json_files)

    for i, json_file in enumerate(json_files):
        with json_file.open() as f:
            json_output = json.load(f)

        df = pd.json_normalize(json_output, 'binding_sites', ['apo_pdb_code', 'holo_pdb_code', 'apo_chain_id', 'holo_chain_id'])
        super_df = pd.concat([super_df, df], ignore_index=True)

        if i % 10 == 0:
                print(f'\rloading... {i} / {total_files} done', end='')

    return super_df

In [ ]:
files = Path(output_root).rglob('*.json')

df = load_df(files)

loading... 1060 / 2542 done

In [ ]:
# je klidne vetsi nez pocet paru, protoze je radek za kazdy ligand, nikoliv pár...
df

In [ ]:
# todo histogram podílu blocked_BS_atoms / BS_atoms

df['blocked_bs_atoms__ratio'] = df['analyses.method4.apo_bs_blocked_atoms__by_blocking_atoms'] / df['atom_count_statistic.binding_atoms_observed_in_apo']

df['bs_residue_count'] = df.residue_ids.apply(len)

df['blocked_bs_residues__ratio'] = df['analyses.method4.apo_bs_blocked_residues__by_blocking_atoms'] / df.bs_residue_count


In [ ]:
df

In [ ]:
import seaborn as sns
# wargs = {'cumulative': True}
# sns.distplot(x, hist_kws=kwargs, kde_kws=kwargs)

sns.ecdfplot(df, x='blocked_bs_atoms__ratio')

In [ ]:
sns.ecdfplot(df, x='blocked_bs_residues__ratio')

todo agregace (binding sites) podle párů, a pak podle apo struktur asi (kolik apo struktur má cryptic binding sites)